In [3]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 25.8 MB/s 
     |████████████████████████████████| 7.6 MB 44.7 MB/s 
     |████████████████████████████████| 1.3 MB 53.6 MB/s 
     |████████████████████████████████| 441 kB 58.9 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 1.9 MB 54.3 MB/s 
     |████████████████████████████████| 9.2 MB 68.7 MB/s 
     |████████████████████████████████| 5.3 MB 53.3 MB/s 
     |████████████████████████████████| 163 kB 72.3 MB/s 
     |████████████████████████████████| 182 kB 66.3 MB/s 
     |████████████████████████████████| 166 kB 60.8 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 166 kB 72.0 MB/s 
     |████████████████████████████████| 162 kB 77.6 MB/s 
     |████████████████████████████████| 162 kB 56.8 MB/s 
     |█████████████████████████████

#Assignment 4 




##Experimental Setup

In [6]:
from google.colab import files
uploaded = files.upload()

Saving hasoc-train.csv to hasoc-train.csv
Saving olid-test.csv to olid-test.csv
Saving olid-train.csv to olid-train.csv
Saving olid-train-small.csv to olid-train-small.csv


In [7]:
import pandas as pd
import io
 
df1 = pd.read_csv(io.BytesIO(uploaded['olid-train-small.csv']))
df2 = pd.read_csv(io.BytesIO(uploaded['olid-test.csv']))
df3 = pd.read_csv(io.BytesIO(uploaded['hasoc-train.csv']))
df4 = pd.read_csv(io.BytesIO(uploaded['olid-train.csv']))
print(df1)
print(df2)
print(df3)
df_val = pd.concat([df1,df4]).drop_duplicates(keep=False)


         id                                               text  labels
0     24590  @USER @USER @USER @USER @USER Hahah a left tel...       0
1     19287  @USER @USER I’m glad you do babe (I kiss you b...       0
2     44676          @USER And I have concerns with”Democrats”       0
3     96110                             @USER AS IT SHOULD BE!       0
4     51557  @USER @USER A horrendous act of course. Conser...       1
...     ...                                                ...     ...
5847  24288  @USER This information is out there and should...       1
5848  13930  #MorningJoe the closer it gets to monday the l...       0
5849  68792  @USER @USER Long time prisoners will also have...       0
5850  19909  @USER @USER It wasn’t that long ago it was tab...       0
5851  33705                    @USER He is so beautiful there.       0

[5852 rows x 3 columns]
        id                                               text  labels
0    15923  #WhoIsQ #WheresTheServer #DumpNike #DECLA

In [4]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

## Prelimnary analysis 

Analysis of different models using correlation

In [2]:
def CalcF1(tp,fp,tn,fn):
  F1 = 2*((tp / (tp + fp)) * (tp/(tp+fn))) / ((tp / (tp + fp)) + (tp/(tp+fn)))
  return F1

###BERT model

In [12]:

model_args_1 = ClassificationArgs()

#Learning rate
model_args_1.learning_rate = 1e-5


#Authorizing the overwriting
model_args_1.overwrite_output_dir = True
model_Bert_1 = ClassificationModel(
    "bert", "bert-base-cased", args=model_args_1
)
model_Bert_1.train_model(df1, output_dir="Bert ID Training 1")


#We evaluate the different models over the dev set
result_Bert_dev, model_outputs_Bert_dev, wrong_predictions_Bert_dev = model_Bert_1.eval_model(df_val)



Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/732 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


  0%|          | 0/7388 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/924 [00:00<?, ?it/s]

In [14]:
F1_Bert_1=CalcF1(result_Bert_dev["tp"] ,result_Bert_dev["fp"] ,result_Bert_dev["tn"] ,result_Bert_dev["fn"] )
print(F1_Bert_1)

0.6530426884650318


###Roberta model

In [15]:
model_args_Roberta = ClassificationArgs()

model_args_Roberta.learning_rate = 1e-5

model_args_Roberta.num_train_epochs = 1
#Authorization of overwriting
model_args_Roberta.overwrite_output_dir = True

#implementation
model_Roberta = ClassificationModel(
    "roberta", "roberta-base", args=model_args_Roberta
)

#training
model_Roberta.train_model(df1, output_dir="Roberta ID Training")

#We evaluate the model
result_Roberta_dev, model_outputs_Roberta_dev, wrong_predictions_Roberta_dev = model_Roberta.eval_model(df_val)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

  0%|          | 0/5852 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/732 [00:00<?, ?it/s]

  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

In [26]:
result_Roberta_dev, model_outputs_Roberta_dev, wrong_predictions_Roberta_dev = model_Roberta.eval_model(df_val)

  0%|          | 0/7388 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/924 [00:00<?, ?it/s]

In [16]:
F1_Roberta=CalcF1(result_Roberta_dev["tp"] ,result_Roberta_dev["fp"] ,result_Roberta_dev["tn"] ,result_Roberta_dev["fn"] )
print(F1_Roberta)

0.6973684210526315


###HateBert

In [17]:
#We assumed that these preliminary results about the Bert model could apply to most of the others 
model_args_HateBert = ClassificationArgs()

#Learning rate
model_args_HateBert.learning_rate = 1e-5
#Nb of epochs
model_args_HateBert.num_train_epochs = 1
#Authorization of overwriting
model_args_HateBert.overwrite_output_dir = True

#implementation
model_HateBert = ClassificationModel(
    "bert", "GroNLP/hateBERT", args=model_args_HateBert
)

#training
model_HateBert.train_model(df1, output_dir="HateBert ID Training")

#We evaluate the model
result_HateBert_dev, model_outputs_HateBert_dev, wrong_predictions_HateBert_dev = model_HateBert.eval_model(df_val)

Downloading:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at GroNLP/hateBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly

Downloading:   0%|          | 0.00/151 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

  0%|          | 0/5852 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/732 [00:00<?, ?it/s]

  0%|          | 0/7388 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/924 [00:00<?, ?it/s]

In [31]:
F1_Bert_1=CalcF1(result_HateBert_dev["tp"] ,result_HateBert_dev["fp"] ,result_HateBert_dev["tn"] ,result_HateBert_dev["fn"] )
print(F1_Bert_1)

0.6058718861209964


###MLCS 

In [19]:
traindata=df1
train_x=traindata['text'].tolist()
train_y=traindata['labels'].tolist()
testdata=pd.read_csv =df_val
test_x=testdata['text'].tolist()
test_y=testdata['labels'].tolist()

text_clf= Pipeline([
    ('vect',CountVectorizer()),
    ('clf', MLPClassifier(max_iter=300))])

clf = text_clf.fit(train_x, train_y)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [28]:
predictions_mlcs_dev = text_clf.predict(df_val['text']).tolist()

print(classification_report(predictions2,df_val['labels']))

              precision    recall  f1-score   support

           0       0.83      0.81      0.82      5343
           1       0.53      0.56      0.54      2045

    accuracy                           0.74      7388
   macro avg       0.68      0.68      0.68      7388
weighted avg       0.75      0.74      0.74      7388



### correlation

In [30]:

bert_prediction_dev = []
hatebert_prediction_dev = []
roberta_prediction_dev = []
for x in model_outputs_Bert_dev:
  if x[0] > x[1]:
    bert_prediction_dev.append(0)
  else:
    bert_prediction_dev.append(1)

for x in model_outputs_Roberta_dev:
  if x[0] > x[1]:
    roberta_prediction_dev.append(0)
  else:
    roberta_prediction_dev.append(1)

for x in model_outputs_HateBert_dev:
  if x[0] > x[1]:
    hatebert_prediction_dev.append(0)
  else:
    hatebert_prediction_dev.append(1)
print(len(bert_prediction_dev))
print(len(hatebert_prediction_dev))
print(len(roberta_prediction_dev))
print(len(predictions_mlcs_dev))

data = {'BERT': bert_prediction_dev,
        'Hate': hatebert_prediction_dev,
        'Roberta': roberta_prediction_dev,
        'MLCS':predictions_mlcs_dev
        }

df = pd.DataFrame(data)

corr_matrix = df.corr()
print(corr_matrix)

7388
7388
7388
7388
             BERT      Hate   Roberta      MLCS
BERT     1.000000  0.626300  0.730565  0.465885
Hate     0.626300  1.000000  0.605966  0.362550
Roberta  0.730565  0.605966  1.000000  0.461262
MLCS     0.465885  0.362550  0.461262  1.000000


##In-domain experiments : 

### Hard and soft majority voting ensemble

Bert

In [43]:
result_Bert_test, model_outputs_Bert_test, wrong_predictions_Bert_test = model_Bert_1.eval_model(df2)
print(model_outputs_Bert_test)

  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

[[ 0.16870117 -0.39746094]
 [-0.22180176  0.06027222]
 [ 0.93896484 -0.85400391]
 ...
 [-0.77246094  0.67431641]
 [ 0.09436035 -0.26318359]
 [ 0.84082031 -1.29785156]]


Roberta

In [45]:
result_Roberta_test, model_outputs_Roberta_test, wrong_predictions_Roberta_test = model_Roberta.eval_model(df2)

  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

Hatebert

In [51]:
result_HateBert_test, model_outputs_HateBert_test, wrong_predictions_HateBert_test = model_HateBert.eval_model(df2)

  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

In [ ]:
#the results of HateBert
F1_HateBert = 2*((result_HateBert["tp"] / (result_HateBert["tp"] + result_HateBert["fp"])) * (result_HateBert["tp"]/(result_HateBert["tp"]+result_HateBert["fn"]))) / ((result_HateBert["tp"] / (result_HateBert["tp"] + result_HateBert["fp"])) + (result_HateBert["tp"]/(result_HateBert["tp"]+result_HateBert["fn"])))
print(F1_HateBert)
print(classification_report(ensemble_df['Soft Label'] ,ensemble_df['Label'] ))

0.6515463917525772


In [52]:
bert_prediction_test = []
hatebert_prediction_test = []
roberta_prediction_test = []
for x in model_outputs_Bert_test:
  if x[0] > x[1]:
    bert_prediction_test.append(0)
  else:
    bert_prediction_test.append(1)

for x in model_outputs_Roberta_test:
  if x[0] > x[1]:
    roberta_prediction_test.append(0)
  else:
    roberta_prediction_test.append(1)

for x in model_outputs_HateBert_test:
  if x[0] > x[1]:
    hatebert_prediction_test.append(0)
  else:
    hatebert_prediction_test.append(1)
print(len(bert_prediction_test))
print(len(hatebert_prediction_test))
print(len(hatebert_prediction_test))


860
7388
7388
7388


MLCS

In [ ]:
predictions_mlcs_test = text_clf.predict(df2['text']).tolist()
predict_proba_mlcs_test =text_clf.predict_proba(df2['text']).tolist()
print(predict_proba_mlcs_test)

combining prediction for Soft ensemble

In [46]:
zero_labels = []
one_labels = []
soft_prediction = []

for x in range(len(model_outputs_Bert_test)):
  current_zero = [model_outputs_Bert_test[x][0],model_outputs_Roberta_test[x][0],predict_proba_mlcs_test[x][0]]
  current_one = [model_outputs_Bert_test[x][1],model_outputs_Roberta_test[x][1],predict_proba_mlcs_test[x][1]]

  zero_labels.append(max(current_zero))
  one_labels.append(max(current_one))

  if zero_labels[x] > one_labels[x]:
    soft_prediction.append(0)
  else:
    soft_prediction.append(1)

soft_prediction
len(soft_prediction)

860

calculating hard ensemble

In [54]:
import numpy as np

ensemble_df = pd.DataFrame(columns=['id','Tweet','Bert_probability','RoBERTa_probability','MLCS_probability','Bert_prediction','RoBERTa_prediction','MLCS_prediction','Label', 'Hard Label', 'Soft Label'])
ensemble_df['Tweet'] = df2['text']
ensemble_df['id'] = df2['id']
ensemble_df['Label'] = df2['labels']
ensemble_df['Bert_probability'] = model_outputs_Bert_test
ensemble_df['RoBERTa_probability'] = model_outputs_Roberta_test
ensemble_df['MLCS_probability'] = predict_proba_mlcs_test
ensemble_df['Bert_prediction'] = bert_prediction_test
ensemble_df['RoBERTa_prediction'] = roberta_prediction_test
ensemble_df['MLCS_prediction'] = predictions_mlcs_test
ensemble_df['Hard Label'] = np.round((ensemble_df['Bert_prediction'] + ensemble_df['RoBERTa_prediction'] + ensemble_df['MLCS_prediction'])/3)
ensemble_df['Soft Label'] = soft_prediction
ensemble_df

,id,Tweet,Bert_probability,RoBERTa_probability,MLCS_probability,Bert_prediction,RoBERTa_prediction,MLCS_prediction,Label,Hard Label,Soft Label
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,0.168701,-0.520020,"[0.0019002193912435184, 0.9980997806087565]",0,1,1,1,1.0,1
1,27014,"#ConstitutionDay is revered by Conservatives, ...",-0.221802,0.754883,"[0.6009497977932312, 0.3990502022067688]",1,0,0,0,0.0,0
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0.938965,0.804199,"[0.9850886135253746, 0.014911386474625405]",0,0,0,0,0.0,0
3,13876,#Watching #Boomer getting the news that she is...,1.247070,1.152344,"[0.9650748029578573, 0.034925197042142656]",0,0,0,0,0.0,0
4,60133,#NoPasaran: Unity demo to oppose the far-right...,0.753906,0.994141,"[0.4542198322155635, 0.5457801677844365]",0,0,1,1,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
855,73439,#DespicableDems lie again about rifles. Dem Di...,0.054504,0.253174,"[0.9348726689567678, 0.06512733104323223]",0,0,0,1,0.0,0
856,25657,#MeetTheSpeakers 🙌 @USER will present in our e...,0.866211,1.361328,"[0.9998067927947489, 0.00019320720525102964]",0,0,0,0,0.0,0
857,67018,3 people just unfollowed me for talking about ...,-0.772461,-1.347656,"[0.7540819616623085, 0.24591803833769155]",1,1,0,1,1.0,1
858,50665,#WednesdayWisdom Antifa calls the right fascis...,0.094360,-0.301758,"[0.019276245029085537, 0.9807237549709145]",0,1,1,0,1.0,1


In [55]:
from sklearn.metrics import classification_report
print("Hard label")
print(classification_report(ensemble_df['Hard Label'] ,ensemble_df['Label'] ))
print("Soft label")
print(classification_report(ensemble_df['Soft Label'] ,ensemble_df['Label'] ))


Hard label
              precision    recall  f1-score   support

         0.0       0.93      0.87      0.90       662
         1.0       0.63      0.77      0.69       198

    accuracy                           0.84       860
   macro avg       0.78      0.82      0.79       860
weighted avg       0.86      0.84      0.85       860

Soft label
              precision    recall  f1-score   support

           0       0.91      0.86      0.89       650
           1       0.63      0.72      0.68       210

    accuracy                           0.83       860
   macro avg       0.77      0.79      0.78       860
weighted avg       0.84      0.83      0.83       860



### Stacking ensemble
creating train data

In [ ]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=e59becaadb625102ba9fcf4530f557bf47ae157339add3f04b72c86b724c3608
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


5 fold cross validation:

In [49]:
from sklearn.model_selection import StratifiedKFold 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import numpy as np
#5FCV output for ensemble 

model_bert_args = ClassificationArgs()
model_roberta_args = ClassificationArgs()
model_hatebert_args = ClassificationArgs()

#Args bert
model_bert_args.learning_rate = 1e-5
model_bert_args.num_train_epochs = 1
model_bert_args.overwrite_output_dir = True

#Args roberta
model_roberta_args.learning_rate = 1e-5
model_roberta_args.num_train_epochs = 1
model_roberta_args.overwrite_output_dir = True

#Args hatebert
model_hatebert_args.learning_rate = 1e-5
model_hatebert_args.num_train_epochs = 1
model_hatebert_args.overwrite_output_dir = True

train = df1

rskf = StratifiedKFold(n_splits=5, shuffle=True)

pred_bert = []
pred_roberta = []
pred_clf = []

idx = []
gold = []
text = []

for train_index, test_index in rskf.split(train['text'], train['labels']):

  train_df = train.iloc[train_index]
  test_df = train.iloc[test_index]

  idx.extend(test_index)
  gold.extend(test_df['labels'])
  text.extend(test_df['text'])

  model_bert = ClassificationModel('bert', 'bert-base-cased', args=model_bert_args)
  
  model_bert.train_model(train_df)
  predictions_bert, prob_bert = model_bert.predict(test_df.text.to_list())
  pred_bert.extend(predictions_bert)

  model_roberta = ClassificationModel("roberta", "roberta-base", args=model_roberta_args)

  model_roberta.train_model(train_df)
  predictions_roberta, prob_roberta = model_roberta.predict(test_df.text.to_list())
  pred_roberta.extend(predictions_roberta)

  text_clf= Pipeline([
    ('vect',CountVectorizer()),
    ('clf', MLPClassifier(max_iter=300))])

  clf2 = text_clf.fit(train_df['text'], train_df['labels'])
  predictions_clf =  text_clf.predict(test_df['text']).tolist()
  pred_clf.extend(predictions_clf)
  # model_hatebert.train_model(train_df)
  # predictions_hatebert, prob_hatebert = model_roberta.predict(test_df.text.to_list())
  # pred_hatebert.extend(predictions_hatebert)




Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/4681 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/586 [00:00<?, ?it/s]

  0%|          | 0/1171 [00:00<?, ?it/s]

...

  0%|          | 0/4682 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/586 [00:00<?, ?it/s]

  0%|          | 0/1170 [00:00<?, ?it/s]

  0%|          | 0/147 [00:00<?, ?it/s]

Combining results

In [50]:
output = pd.DataFrame(columns = ['text', 'bert', 'roberta', 'mlcs', 'label']) 
output['text'] = text
output['bert'] = pred_bert
output['roberta'] = pred_roberta
output['mlcs'] = pred_clf
output['label'] = gold
print(output)

                                                   text  bert  roberta  mlcs  \
0                              @USER Another uninformed     0        0     0   
1     @USER We've got a big story coming out (most p...     0        0     0   
2     @USER  How did Britain end up upside down in t...     1        1     1   
3     @USER @USER @USER @USER Hes comin for ur wooll...     1        1     1   
4           @USER man can we hang out you are hilarious     0        0     0   
...                                                 ...   ...      ...   ...   
5847  @USER but i dont think all of us are depressed...     1        1     0   
5848  #MAGA! IF YOU ARE ALL IN FOR #TRUMP FOLLOW ME ...     0        0     0   
5849  @USER me trying to avoid the freshman boys who...     0        0     0   
5850  @USER @USER Thanks, I was wondering what this ...     0        0     0   
5851  @USER This information is out there and should...     0        0     1   

      label  
0         0  
1         0

In [51]:
#Add features 

#Add number of characters of each sentence
num_char = [len(i) for i in output['text']]
output['num_char'] = num_char

num_words = [len(i.split()) for i in output['text']]
output['num_words'] = num_words
output

,text,bert,roberta,mlcs,label,num_char,num_words
0,@USER Another uninformed,0,0,0,0,24,3
1,@USER We've got a big story coming out (most p...,0,0,0,0,132,25
2,@USER How did Britain end up upside down in t...,1,1,1,0,223,38
3,@USER @USER @USER @USER Hes comin for ur wooll...,1,1,1,1,55,10
4,@USER man can we hang out you are hilarious,0,0,0,0,43,9
...,...,...,...,...,...,...,...
5847,@USER but i dont think all of us are depressed...,1,1,0,0,213,44
5848,#MAGA! IF YOU ARE ALL IN FOR #TRUMP FOLLOW ME ...,0,0,0,0,69,15
5849,@USER me trying to avoid the freshman boys who...,0,0,0,1,94,20
5850,"@USER @USER Thanks, I was wondering what this ...",0,0,0,0,207,33


Train meta model

In [61]:
from joblib.logger import print_function
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
X_train = output[['bert','mlcs','mlcs','num_char','num_words']].to_numpy()
Y_train = output['label']

clf = LogisticRegression()
clf.fit(X_train, Y_train)



LogisticRegression()

Test meta model

In [1]:

test_df = df2
text=test_df['text']
gold=test_df['labels']
# predict
predictions_bert, prob_bert = model_bert.predict(test_df.text.to_list())
predictions_roberta, prob_roberta = model_roberta.predict(test_df.text.to_list())
predictions_clf =  text_clf.predict(test_df['text']).tolist()

outputtest = pd.DataFrame(columns = ['text', 'bert', 'roberta', 'mlcs', 'label']) 
outputtest['text'] = text
outputtest['bert'] = predictions_bert
outputtest['roberta'] = predictions_roberta
outputtest['mlcs'] = predictions_clf
outputtest['label'] = gold
# add features
num_char = [len(i) for i in outputtest['text']]
outputtest['num_char'] = num_char

num_words = [len(i.split()) for i in outputtest['text']]
outputtest['num_words'] = num_words
X_test = outputtest[['bert','mlcs','mlcs','num_char','num_words']].to_numpy()
print(X_test)
y_pred = clf.predict(X_test)


print(classification_report(outputtest['label'],y_pred))


NameError: ignored

## Cross domain

Training models

In [10]:
model_args_1 = ClassificationArgs()

#BERT

#Learning rate
model_args_1.learning_rate = 1e-5
model_args_1.num_train_epochs = 1
#Authorizing the overwriting
model_args_1.overwrite_output_dir = True
model_Bert_1 = ClassificationModel(
    "bert", "bert-base-cased", args=model_args_1
)
model_Bert_1.train_model(df3, output_dir="Bert ID Training 1")

#ROBERTA

model_args_Roberta = ClassificationArgs()
model_args_Roberta.learning_rate = 1e-5
model_args_Roberta.num_train_epochs = 1
#Authorization of overwriting
model_args_Roberta.overwrite_output_dir = True
#implementation
model_Roberta = ClassificationModel(
    "roberta", "roberta-base", args=model_args_Roberta
)
#training
model_Roberta.train_model(df3, output_dir="Roberta ID Training")

#MLCS
traindata=df3
train_x=traindata['text'].tolist()
train_y=traindata['labels'].tolist()
testdata=pd.read_csv =df_val
test_x=testdata['text'].tolist()
test_y=testdata['labels'].tolist()

text_clf= Pipeline([
    ('vect',CountVectorizer()),
    ('clf', MLPClassifier(max_iter=300))])

clf = text_clf.fit(train_x, train_y)



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/5852 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/732 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

  0%|          | 0/5852 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/732 [00:00<?, ?it/s]

Testing models

In [12]:
result_Bert_test, model_outputs_Bert_test, wrong_predictions_Bert_test = model_Bert_1.eval_model(df2)
result_Roberta_test, model_outputs_Roberta_test, wrong_predictions_Roberta_test = model_Roberta.eval_model(df2)
predictions_mlcs_test = text_clf.predict(df2['text']).tolist()
predict_proba_mlcs_test =text_clf.predict_proba(df2['text']).tolist()
bert_prediction_test = []
hatebert_prediction_test = []
roberta_prediction_test = []
for x in model_outputs_Bert_test:
  if x[0] > x[1]:
    bert_prediction_test.append(0)
  else:
    bert_prediction_test.append(1)

for x in model_outputs_Roberta_test:
  if x[0] > x[1]:
    roberta_prediction_test.append(0)
  else:
    roberta_prediction_test.append(1)



  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

SOFT majority

In [13]:
zero_labels = []
one_labels = []
soft_prediction = []

for x in range(len(model_outputs_Bert_test)):
  current_zero = [model_outputs_Bert_test[x][0],model_outputs_Roberta_test[x][0],predict_proba_mlcs_test[x][0]]
  current_one = [model_outputs_Bert_test[x][1],model_outputs_Roberta_test[x][1],predict_proba_mlcs_test[x][1]]

  zero_labels.append(max(current_zero))
  one_labels.append(max(current_one))

  if zero_labels[x] > one_labels[x]:
    soft_prediction.append(0)
  else:
    soft_prediction.append(1)

soft_prediction

[0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,


HARD prediction + final result

In [14]:
import numpy as np

ensemble_df = pd.DataFrame(columns=['id','Tweet','Bert_probability','RoBERTa_probability','MLCS_probability','Bert_prediction','RoBERTa_prediction','MLCS_prediction','Label', 'Hard Label', 'Soft Label'])
ensemble_df['Tweet'] = df2['text']
ensemble_df['id'] = df2['id']
ensemble_df['Label'] = df2['labels']
ensemble_df['Bert_probability'] = model_outputs_Bert_test
ensemble_df['RoBERTa_probability'] = model_outputs_Roberta_test
ensemble_df['MLCS_probability'] = predict_proba_mlcs_test
ensemble_df['Bert_prediction'] = bert_prediction_test
ensemble_df['RoBERTa_prediction'] = roberta_prediction_test
ensemble_df['MLCS_prediction'] = predictions_mlcs_test
ensemble_df['Hard Label'] = np.round((ensemble_df['Bert_prediction'] + ensemble_df['RoBERTa_prediction'] + ensemble_df['MLCS_prediction'])/3)
ensemble_df['Soft Label'] = soft_prediction
ensemble_df

,id,Tweet,Bert_probability,RoBERTa_probability,MLCS_probability,Bert_prediction,RoBERTa_prediction,MLCS_prediction,Label,Hard Label,Soft Label
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,0.173584,0.083374,"[0.9832935048196583, 0.01670649518034175]",0,1,0,1,0.0,0
1,27014,"#ConstitutionDay is revered by Conservatives, ...",0.358398,0.525879,"[0.9674248560724884, 0.03257514392751159]",0,0,0,0,0.0,0
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0.306641,0.621582,"[0.9999896091820147, 1.0390817985264296e-05]",0,0,0,0,0.0,0
3,13876,#Watching #Boomer getting the news that she is...,-0.066040,0.870117,"[1.7213655026937502e-06, 0.9999982786344973]",1,0,1,0,1.0,1
4,60133,#NoPasaran: Unity demo to oppose the far-right...,0.340820,0.934570,"[0.9999629809792809, 3.7019020719055275e-05]",0,0,0,1,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
855,73439,#DespicableDems lie again about rifles. Dem Di...,0.165405,0.665039,"[0.9999232574358777, 7.674256412222812e-05]",0,0,0,1,0.0,0
856,25657,#MeetTheSpeakers 🙌 @USER will present in our e...,0.412598,1.060547,"[0.9996175997730523, 0.00038240022694772306]",0,0,0,0,0.0,0
857,67018,3 people just unfollowed me for talking about ...,-0.075439,0.183105,"[0.9999983951104918, 1.6048895082203403e-06]",1,1,0,1,1.0,0
858,50665,#WednesdayWisdom Antifa calls the right fascis...,0.214478,0.090088,"[0.030965207109457604, 0.9690347928905424]",0,1,1,0,1.0,1


In [15]:
from sklearn.metrics import classification_report
print("Hard label")
print(classification_report(ensemble_df['Hard Label'] ,ensemble_df['Label'] ))
print("Soft label")
print(classification_report(ensemble_df['Soft Label'] ,ensemble_df['Label'] ))

Hard label
              precision    recall  f1-score   support

         0.0       0.91      0.82      0.86       686
         1.0       0.49      0.68      0.57       174

    accuracy                           0.79       860
   macro avg       0.70      0.75      0.72       860
weighted avg       0.83      0.79      0.80       860

Soft label
              precision    recall  f1-score   support

           0       0.77      0.80      0.78       593
           1       0.51      0.46      0.48       267

    accuracy                           0.69       860
   macro avg       0.64      0.63      0.63       860
weighted avg       0.69      0.69      0.69       860



Error analysis for hard majority 

In [1]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       'display.max_colwidth', -1
                       ): 
                       print(ensemble_df.loc[ensemble_df['Hard Label'] != ensemble_df['Label']]['Tweet','Hard label','Label'])


NameError: ignored

stacking


In [16]:
from sklearn.model_selection import StratifiedKFold 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import numpy as np
#5FCV output for ensemble 

model_bert_args = ClassificationArgs()
model_roberta_args = ClassificationArgs()
model_hatebert_args = ClassificationArgs()

#Args bert
model_bert_args.learning_rate = 1e-5
model_bert_args.num_train_epochs = 1
model_bert_args.overwrite_output_dir = True

#Args roberta
model_roberta_args.learning_rate = 1e-5
model_roberta_args.num_train_epochs = 1
model_roberta_args.overwrite_output_dir = True

#Args hatebert
model_hatebert_args.learning_rate = 1e-5
model_hatebert_args.num_train_epochs = 1
model_hatebert_args.overwrite_output_dir = True

train = df3

rskf = StratifiedKFold(n_splits=5, shuffle=True)

pred_bert = []
pred_roberta = []
pred_clf = []

idx = []
gold = []
text = []

for train_index, test_index in rskf.split(train['text'], train['labels']):

  train_df = train.iloc[train_index]
  test_df = train.iloc[test_index]

  idx.extend(test_index)
  gold.extend(test_df['labels'])
  text.extend(test_df['text'])

  model_bert = ClassificationModel('bert', 'bert-base-cased', args=model_bert_args)
  
  model_bert.train_model(train_df)
  predictions_bert, prob_bert = model_bert.predict(test_df.text.to_list())
  pred_bert.extend(predictions_bert)

  model_roberta = ClassificationModel("roberta", "roberta-base", args=model_roberta_args)

  model_roberta.train_model(train_df)
  predictions_roberta, prob_roberta = model_roberta.predict(test_df.text.to_list())
  pred_roberta.extend(predictions_roberta)

  text_clf= Pipeline([
    ('vect',CountVectorizer()),
    ('clf', MLPClassifier(max_iter=300))])

  clf2 = text_clf.fit(train_df['text'], train_df['labels'])
  predictions_clf =  text_clf.predict(test_df['text']).tolist()
  pred_clf.extend(predictions_clf)
output = pd.DataFrame(columns = ['text', 'bert', 'roberta', 'mlcs', 'label']) 
output['text'] = text
output['bert'] = pred_bert
output['roberta'] = pred_roberta
output['mlcs'] = pred_clf
output['label'] = gold

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/4681 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/586 [00:00<?, ?it/s]

  0%|          | 0/1171 [00:00<?, ?it/s]

...

  0%|          | 0/4682 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/586 [00:00<?, ?it/s]

  0%|          | 0/1170 [00:00<?, ?it/s]

  0%|          | 0/147 [00:00<?, ?it/s]

In [18]:
from joblib.logger import print_function
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
X_train = output[['bert','mlcs','mlcs','num_char','num_words']].to_numpy()
Y_train = output['label']

clf = LogisticRegression()
clf.fit(X_train, Y_train)

LogisticRegression()

Predict test data with stacking ensemble 

In [19]:
test_df = df2
text=test_df['text']
gold=test_df['labels']
# predict
predictions_bert, prob_bert = model_bert.predict(test_df.text.to_list())
predictions_roberta, prob_roberta = model_roberta.predict(test_df.text.to_list())
predictions_clf =  text_clf.predict(test_df['text']).tolist()

outputtest = pd.DataFrame(columns = ['text', 'bert', 'roberta', 'mlcs', 'label','prediction']) 
outputtest['text'] = text
outputtest['bert'] = predictions_bert
outputtest['roberta'] = predictions_roberta
outputtest['mlcs'] = predictions_clf
outputtest['label'] = gold
# add features
num_char = [len(i) for i in outputtest['text']]
outputtest['num_char'] = num_char

num_words = [len(i.split()) for i in outputtest['text']]
outputtest['num_words'] = num_words
X_test = outputtest[['bert','mlcs','mlcs','num_char','num_words']].to_numpy()
print(X_test)
y_pred = clf.predict(X_test)
outputtest['prediction'] = gold

print(classification_report(outputtest['label'],y_pred))

  0%|          | 0/860 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/860 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

[[  0   0   0 245  27]
 [  0   0   0 111  13]
 [  0   0   0 204  28]
 ...
 [  1   0   0 205  37]
 [  0   1   1 267  42]
 [  0   0   0  45   6]]
              precision    recall  f1-score   support

           0       0.79      0.82      0.80       620
           1       0.48      0.42      0.45       240

    accuracy                           0.71       860
   macro avg       0.63      0.62      0.62       860
weighted avg       0.70      0.71      0.70       860

